In [ ]:
!git clone https://github.com/LucasBorges4/metaheuristicas.git

Cloning into 'metaheuristicas'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 3.19 MiB | 14.66 MiB/s, done.


In [ ]:
!unzip metaheuristicas/ep.zip -d metaheuristicas

Archive:  metaheuristicas/ep.zip
   creating: metaheuristicas/instances/
  inflating: metaheuristicas/instances/Readme.md  
  inflating: metaheuristicas/instances/ep04.dat  
  inflating: metaheuristicas/instances/ep01.dat  
  inflating: metaheuristicas/instances/ep03.dat  
  inflating: metaheuristicas/instances/ep05.dat  
  inflating: metaheuristicas/instances/ep02.dat  
  inflating: metaheuristicas/instances/ep09.dat  
  inflating: metaheuristicas/instances/ep08.dat  
  inflating: metaheuristicas/instances/ep06.dat  
  inflating: metaheuristicas/instances/ep10.dat  
  inflating: metaheuristicas/instances/ep07.dat  


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import time
from collections import defaultdict

class EvolutionaryDanceEP:
    def __init__(self, instance,
                 # Parâmetros principais
                 pop_size=200,
                 mutation_rate=0.2,
                 alpha=0.31,
                 max_iter=1000,

                 # Novos parâmetros para controle de status
                 status_decay_rate=0.05,           # Taxa de decremento do status
                 low_status_threshold=0.5,          # Limite para considerar baixo status
                 status_penalty_exponent=2,         # Expoente para penalização de status

                 # Parâmetros biológicos
                 max_mates_per_female=3,
                 low_status_mating_prob=0.15,
                 reverse_chimerism_prob=0.25):

        # Armazena os novos parâmetros
        self.status_decay_rate = status_decay_rate
        self.low_status_threshold = low_status_threshold
        self.status_penalty_exponent = status_penalty_exponent
        # Parâmetros do problema
        self.instance = instance
        self.n = instance['n']
        self.W = instance['W']
        self.w = np.array(instance['w'])
        self.t = np.array(instance['t'])
        self.neighbors = self.build_neighbors()

        # Novos parâmetros biológicos
        self.max_mates_per_female = max_mates_per_female
        self.low_status_mating_prob = low_status_mating_prob
        self.reverse_chimerism_prob = reverse_chimerism_prob

        # Parâmetros do algoritmo
        self.pop_size = pop_size
        self.mutation_rate = mutation_rate
        self.alpha = alpha  # Taxa de microquimerismo
        self.max_iter = max_iter

        # Inicialização da população
        self.population = self.initialize_population()
        self.gender = np.array(['F'] * (pop_size // 2) + ['M'] * (pop_size // 2))
        self.status = np.zeros(pop_size)  # Nível de status/recurso
        self.dance_style = np.random.rand(pop_size, 5)  # Padrões de dança
        self.chimeric_dna = [set() for _ in range(pop_size)]  # Rastreamento de microquimerismo
        self.epigenetic_marks = [np.zeros(self.n) for _ in range(pop_size)]  # Marcas epigenéticas
        self.fitness_history = []
        self.best_solution = None
        self.best_fitness = -np.inf
        self.mate_counts = []  # Para armazenar estatísticas de acasalamento

        # Inicializar atributos
        self.initialize_attributes()

    def build_neighbors(self):
        """Constrói lista de vizinhos para cada ingrediente"""
        neighbors = [[] for _ in range(self.n)]
        for j, k in self.instance['incompatible_pairs']:
            neighbors[j].append(k)
            neighbors[k].append(j)
        return neighbors

    def initialize_population(self):
        """Inicializa população com soluções viáveis"""
        population = []
        while len(population) < self.pop_size:
            sol = np.zeros(self.n, dtype=int)
            weight = 0

            # Adiciona ingredientes aleatórios respeitando restrições
            indices = list(range(self.n))
            random.shuffle(indices)

            for i in indices:
                if weight + self.w[i] > self.W:
                    continue

                # Verifica conflitos
                conflict = False
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        conflict = True
                        break

                if not conflict and random.random() < 0.7:
                    sol[i] = 1
                    weight += self.w[i]

            population.append(sol)
        return np.array(population)

    def initialize_attributes(self):
        """Inicializa status e dança com base na solução"""
        # Calcula fitness para cada indivíduo
        fitness = []
        for sol in self.population:
            flavor = np.sum(self.t * sol)
            weight = np.sum(self.w * sol)

            # Penaliza soluções inviáveis
            penalty = 0
            if weight > self.W:
                penalty = (weight - self.W) * 100

            for i in range(self.n):
                if sol[i] == 1:
                    for j in self.neighbors[i]:
                        if sol[j] == 1:
                            penalty += 100
                            break

            fitness.append(flavor - penalty)

        fitness = np.array(fitness)
        self.status = (fitness - np.min(fitness)) / (np.max(fitness) - np.min(fitness) + 1e-10)

        # Atualiza melhor solução
        max_idx = np.argmax(fitness)
        if fitness[max_idx] > self.best_fitness:
            self.best_fitness = fitness[max_idx]
            self.best_solution = self.population[max_idx].copy()

        # Estilo de dança baseado na solução
        for i in range(self.pop_size):
            sol = self.population[i]
            # Características: proporção de ingredientes selecionados em grupos
            group_size = max(1, self.n // 5)
            for g in range(5):
                start = g * group_size
                end = min((g+1) * group_size, self.n)
                self.dance_style[i, g] = np.mean(sol[start:end])

    def evaluate_fitness(self):
        """Calcula fitness baseado na função objetivo e penalidades"""
        fitness = []
        for i, sol in enumerate(self.population):
            flavor = np.sum(self.t * sol)
            weight = np.sum(self.w * sol)

            # Penaliza soluções inviáveis
            penalty = 0
            if weight > self.W:
                penalty = (weight - self.W) * 100

            # Verifica incompatibilidades
            for j in range(self.n):
                if sol[j] == 1:
                    for k in self.neighbors[j]:
                        if sol[k] == 1:
                            penalty += 100
                            break

            # Aplica efeito epigenético (genes ativados têm maior contribuição)
            if i < len(self.epigenetic_marks):  # Verificação de segurança
                activated_flavor = np.sum(self.t * sol * (1 + 0.2 * self.epigenetic_marks[i]))
                flavor = max(flavor, activated_flavor)

            fitness.append(flavor - penalty)

        fitness = np.array(fitness)

        # Atualiza melhor solução global
        max_idx = np.argmax(fitness)
        if fitness[max_idx] > self.best_fitness:
            self.best_fitness = fitness[max_idx]
            self.best_solution = self.population[max_idx].copy()

        self.fitness_history.append(np.max(fitness))
        return fitness

    def female_dance_attraction(self, female_id, male_ids):
        """Calcula atração gerada pela dança da fêmea"""
        female_dance = self.dance_style[female_id]
        reactions = []

        for mid in male_ids:
            # Verificação de segurança para índices válidos
            if mid >= len(self.dance_style) or mid < 0:
                reactions.append(0)
                continue

            # Macho responde à dança baseado em seu status
            male_status = self.status[mid]

            # Similaridade de movimento (distância cosseno)
            male_dance = self.dance_style[mid]
            norm_female = np.linalg.norm(female_dance)
            norm_male = np.linalg.norm(male_dance)

            if norm_female > 1e-10 and norm_male > 1e-10:
                dance_similarity = np.dot(female_dance, male_dance) / (norm_female * norm_male)
            else:
                dance_similarity = 0.0

            # Machos de alto status são mais exigentes
            attraction = male_status * dance_similarity

            # Adicionar componente aleatória (10-20%)
            randomness = 0.8 + 0.4 * random.random()
            reactions.append(attraction * randomness)

        return np.array(reactions)

    def mate_selection(self):
      """Seleção hierárquica com fêmeas podendo ter múltiplos parceiros"""
      female_ids = np.where(self.gender == 'F')[0]
      male_ids = np.where(self.gender == 'M')[0]

      # Filtra índices válidos
      female_ids = female_ids[(female_ids >= 0) & (female_ids < self.pop_size)]
      male_ids = male_ids[(male_ids >= 0) & (male_ids < self.pop_size)]

      if len(female_ids) == 0 or len(male_ids) == 0:
          return []

      # Fêmeas dançam para todos os machos simultaneamente
      attraction_matrix = np.zeros((len(female_ids), len(male_ids)))
      for i, fid in enumerate(female_ids):
          attraction_matrix[i] = self.female_dance_attraction(fid, male_ids)

      # Machos escolhem fêmeas baseado na atração percebida
      pairs = []
      female_mate_counts = defaultdict(int)  # Contagem de parceiros por fêmea

      # Machos de alto status escolhem primeiro
      male_status = self.status[male_ids]
      sorted_male_indices = np.argsort(male_status)[::-1]

      for local_midx in sorted_male_indices:
          mid = male_ids[local_midx]  # ID global do macho

          # Fêmeas disponíveis (que ainda não atingiram o máximo de parceiros)
          available_females = [i for i, fid in enumerate(female_ids)
                              if female_mate_counts[fid] < self.max_mates_per_female]

          if not available_females:
              continue

          # Escolhe a fêmea mais atraente disponível
          attraction_values = attraction_matrix[available_females, local_midx]
          best_fidx_in_available = np.argmax(attraction_values)
          fid = female_ids[available_females[best_fidx_in_available]]

          pairs.append((fid, mid))
          female_mate_counts[fid] += 1

      # Fêmeas podem escolher machos de baixo status adicionalmente
      for fid in female_ids:
          if random.random() < self.low_status_mating_prob and female_mate_counts[fid] < self.max_mates_per_female:
              # Seleciona machos de baixo status não pareados com esta fêmea
              current_mates = [m for f, m in pairs if f == fid]
              low_status_males = [mid for mid in male_ids
                                  if mid not in current_mates and self.status[mid] < 0.5]

              # Penalização baseada no status: quanto menor o status, menor a chance de ser escolhido
              if low_status_males:
                  # Calcula probabilidades inversamente proporcionais ao status
                  status_values = [self.status[mid] for mid in low_status_males]
                  min_status = min(status_values)

                  # Penalização exponencial: machos com status muito baixo têm chance drasticamente reduzida
                  probabilities = [(self.status[mid] - min_status + 0.1) ** 2 for mid in low_status_males]
                  total = sum(probabilities)

                  if total > 0:
                      # Normaliza as probabilidades
                      probabilities = [p / total for p in probabilities]
                      chosen_male = np.random.choice(low_status_males, p=probabilities)
                  else:
                      chosen_male = random.choice(low_status_males)

                  pairs.append((fid, chosen_male))
                  female_mate_counts[fid] += 1

                  # Decrementa o status do macho escolhido (penalização por acasalar com baixo status)
                  self.status[chosen_male] = max(0, self.status[chosen_male] - 0.05)

      return pairs

    def microchimerism(self, recipient_id, donor_id):
        """Transferência de DNA entre indivíduos"""
        # Verificação de índices válidos
        if recipient_id >= self.pop_size or donor_id >= self.pop_size:
            return

        # Selecionar genes para transferência
        transfer_count = max(1, int(self.alpha * self.n))
        gene_idx = random.sample(range(self.n), transfer_count)

        # Incorporar genes ao DNA do recipiente
        for idx in gene_idx:
            self.population[recipient_id][idx] = self.population[donor_id][idx]
            # Atualizar marca epigenética (50% de chance de herdar a marca)
            if random.random() < 0.5:
                self.epigenetic_marks[recipient_id][idx] = self.epigenetic_marks[donor_id][idx]

        # Registrar microquimerismo
        self.chimeric_dna[recipient_id].update(gene_idx)

    def reverse_microchimerism(self, female_id, male_id):
        """Fêmea edita genes de machos de baixo status"""
        # Verificação de índices válidos
        if female_id >= self.pop_size or male_id >= self.pop_size:
            return

        # Genes que a fêmea pode transferir para o macho
        transfer_count = max(1, int(self.alpha * self.n))
        gene_idx = random.sample(range(self.n), transfer_count)

        for idx in gene_idx:
            # A fêmea transfere seu gene com 60% de chance
            if random.random() < 0.6:
                self.population[male_id][idx] = self.population[female_id][idx]
                # Herança epigenética mais forte (70% de chance)
                if random.random() < 0.7:
                    self.epigenetic_marks[male_id][idx] = self.epigenetic_marks[female_id][idx]

        # Registrar no macho
        self.chimeric_dna[male_id].update(gene_idx)

    def repair_solution(self, sol):
        """Repara solução para torná-la viável usando abordagem gulosa"""
        weight = np.sum(self.w * sol)

        # Lista de ingredientes selecionados
        selected = np.where(sol == 1)[0]

        # Remove ingredientes se peso exceder capacidade
        while weight > self.W and len(selected) > 0:
            # Encontra o ingrediente com menor razão sabor/peso
            min_ratio = float('inf')
            idx_to_remove = -1
            for i in selected:
                ratio = self.t[i] / self.w[i] if self.w[i] > 0 else float('inf')
                if ratio < min_ratio:
                    min_ratio = ratio
                    idx_to_remove = i

            sol[idx_to_remove] = 0
            weight -= self.w[idx_to_remove]
            selected = np.setdiff1d(selected, [idx_to_remove])

        # Remove conflitos de incompatibilidade
        for i in range(self.n):
            if sol[i] == 1:
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        # Remove o que tem menor sabor
                        if self.t[i] < self.t[j]:
                            sol[i] = 0
                        else:
                            sol[j] = 0
                        break

        # Tenta adicionar ingredientes sem conflitos (busca local)
        indices = list(range(self.n))
        # Ordena por sabor/peso decrescente
        indices.sort(key=lambda i: self.t[i] / self.w[i] if self.w[i] > 0 else 0, reverse=True)

        for i in indices:
            if sol[i] == 0 and weight + self.w[i] <= self.W:
                conflict = False
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        conflict = True
                        break
                if not conflict:
                    sol[i] = 1
                    weight += self.w[i]

        return sol

    def epigenetic_inheritance(self, mother, father, child_dna):
        """Herança epigenética com influência parental"""
        # Herança: 70% de chance de herdar a marca da mãe, 30% do pai
        if mother < len(self.epigenetic_marks) and father < len(self.epigenetic_marks):
            if random.random() < 0.7:
                child_epigenetic = self.epigenetic_marks[mother].copy()
            else:
                child_epigenetic = self.epigenetic_marks[father].copy()
        else:
            child_epigenetic = np.zeros(self.n)

        # Modificação epigenética baseada no ambiente (solução)
        for i in range(self.n):
            if child_dna[i] == 1 and random.random() < 0.3:
                child_epigenetic[i] = min(1.0, child_epigenetic[i] + 0.2)
            elif child_dna[i] == 0 and random.random() < 0.2:
                child_epigenetic[i] = max(0.0, child_epigenetic[i] - 0.1)

        return child_epigenetic

    def reproduce(self, pairs):
        new_population = []
        new_chimeric = []
        new_epigenetic = []
        new_gender = []

        # Cada par pode produzir filhos
        for mother, father in pairs:
            for _ in range(2):  # Cada par produz 2 filhos
                # Determinar gênero
                gender = 'F' if random.random() < 0.5 else 'M'
                new_gender.append(gender)

                # Herança genética (crossover uniforme com viés materno)
                child_dna = np.zeros(self.n, dtype=int)
                for i in range(self.n):
                    if random.random() < 0.6:  # 60% de chance de herdar da mãe
                        child_dna[i] = self.population[mother][i]
                    else:
                        child_dna[i] = self.population[father][i]

                # Microquimerismo materno
                if mother < len(self.chimeric_dna) and self.chimeric_dna[mother]:
                    chimeric_genes = random.sample(
                        list(self.chimeric_dna[mother]),
                        min(3, len(self.chimeric_dna[mother])))
                    for gene in chimeric_genes:
                        child_dna[gene] = self.population[mother][gene]

                # Mutação
                for i in range(self.n):
                    if random.random() < self.mutation_rate:
                        child_dna[i] = 1 - child_dna[i]

                # Repara solução
                child_dna = self.repair_solution(child_dna)

                new_population.append(child_dna)
                new_chimeric.append(set())

                # Herança epigenética
                child_epigenetic = self.epigenetic_inheritance(mother, father, child_dna)
                new_epigenetic.append(child_epigenetic)

        # Preencher população restante com as melhores soluções
        if len(new_population) < self.pop_size:
            fitness = self.evaluate_fitness()
            best_indices = np.argsort(fitness)[-(self.pop_size - len(new_population)):]
            for idx in best_indices:
                if idx < len(self.population):
                    new_population.append(self.population[idx].copy())
                    new_chimeric.append(self.chimeric_dna[idx].copy() if idx < len(self.chimeric_dna) else set())
                    new_gender.append(self.gender[idx])
                    new_epigenetic.append(self.epigenetic_marks[idx].copy() if idx < len(self.epigenetic_marks) else np.zeros(self.n))

        # Atualizar população
        self.population = np.array(new_population)
        self.chimeric_dna = new_chimeric
        self.gender = np.array(new_gender)
        self.epigenetic_marks = new_epigenetic

    def evolve(self):
        start_time = time.time()
        status_diff = []
        chimeric_levels = []

        for gen in range(self.max_iter):
            fitness = self.evaluate_fitness()
            pairs = self.mate_selection()

            # Coletar estatísticas de acasalamento
            female_mates = defaultdict(int)
            for f, m in pairs:
                female_mates[f] += 1
            if female_mates:
                self.mate_counts.append(np.mean(list(female_mates.values())))
            else:
                self.mate_counts.append(0)

            # Aplicar microquimerismo e microquimerismo reverso
            for mother, father in pairs:
                # Microquimerismo tradicional (macho -> fêmea)
                self.microchimerism(mother, father)

                # Microquimerismo reverso (fêmea -> macho) para machos de baixo status
                if father < len(self.status) and self.status[father] < 0.4 and random.random() < self.reverse_chimerism_prob:
                    self.reverse_microchimerism(mother, father)

            self.reproduce(pairs)
            self.initialize_attributes()  # Atualizar status e dança

            # Coletar dados para análise
            if pairs:
                f_status = [self.status[f] for f, _ in pairs if f < len(self.status)]
                m_status = [self.status[m] for _, m in pairs if m < len(self.status)]
                status_diff.append(np.mean(m_status) - np.mean(f_status) if f_status and m_status else 0)
            else:
                status_diff.append(0)

            # Calcular nível de microquimerismo
            if self.chimeric_dna:
                chimeric_levels.append(np.mean([len(dna) for dna in self.chimeric_dna if dna is not None]) / self.n)
            else:
                chimeric_levels.append(0)

            # Log de progresso
            if gen % 10 == 0:
                print(f"Gen {gen}: Best Fitness={self.best_fitness:.1f} | "
                      f"Status Diff={status_diff[-1]:.3f} | "
                      f"Chimerism={chimeric_levels[-1]:.3f} | "
                      f"Avg Mates={self.mate_counts[-1]:.2f}")

        # Resultado final
        elapsed = time.time() - start_time
        flavor = np.sum(self.t * self.best_solution)
        weight = np.sum(self.w * self.best_solution)

        print(f"\nMelhor solução encontrada em {elapsed:.2f}s:")
        print(f"  Sabor total: {flavor}")
        print(f"  Peso total: {weight}/{self.W}")
        print(f"  Fitness: {self.best_fitness:.1f}")
        print(f"  Mates per female: {np.mean(self.mate_counts):.2f}")

        return self.best_solution, flavor

    def plot_results(self):
        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(self.fitness_history, 'g-')
        plt.title('Evolução da Aptidão Máxima')
        plt.xlabel('Geração')
        plt.ylabel('Fitness')
        plt.grid(alpha=0.3)

        plt.subplot(2, 2, 2)
        if self.chimeric_dna:
            chimeric_levels = [len(dna)/self.n for dna in self.chimeric_dna if dna is not None]
            plt.hist(chimeric_levels, bins=20, color='blue', alpha=0.7) if chimeric_levels else None
            plt.title('Distribuição de Microquimerismo')
            plt.xlabel('Frações de DNA Incorporado')
            plt.ylabel('Frequência')

        plt.subplot(2, 2, 3)
        female_indices = np.where(self.gender == 'F')[0]
        male_indices = np.where(self.gender == 'M')[0]
        if female_indices.size > 0 and male_indices.size > 0:
            plt.scatter(self.status[female_indices], self.status[male_indices], alpha=0.6)
            plt.title('Correlação de Status entre Gêneros')
            plt.xlabel('Status Feminino')
            plt.ylabel('Status Masculino')

        plt.subplot(2, 2, 4)
        if self.mate_counts:
            plt.hist(self.mate_counts, bins=np.arange(0, self.max_mates_per_female+2)-0.5,
                    rwidth=0.8, color='purple')
            plt.title('Distribuição de Parceiros por Fêmea')
            plt.xlabel('Número de Parceiros')
            plt.ylabel('Frequência')

        plt.tight_layout()
        plt.savefig('evolutionary_dance_ep_results.png')
        plt.show()

# Função para ler instâncias
def read_ep_instance(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    blocks = []
    current_block = []
    for line in lines:
        if not line:
            if current_block:
                blocks.append(current_block)
                current_block = []
        else:
            current_block.append(line)
    if current_block:
        blocks.append(current_block)

    n, num_incompatible, W = map(int, blocks[0][0].split())

    t = []
    for line in blocks[1]:
        t.extend(map(int, line.split()))

    w = []
    for line in blocks[2]:
        w.extend(map(int, line.split()))

    incompatible_pairs = []
    if len(blocks) >= 4:
        for line in blocks[3]:
            j, k = map(int, line.split())
            incompatible_pairs.append((j-1, k-1))

    return {
        'n': n,
        'num_incompatible': num_incompatible,
        'W': W,
        't': t,
        'w': w,
        'incompatible_pairs': incompatible_pairs
    }

# Experimento principal com novos parâmetros
def run_evolutionary_dance_experiment(instances_dir, output_dir, runs=10):
    instance_files = sorted([f for f in os.listdir(instances_dir) if f.endswith('.dat')])
    bkv = {
        'ep01': 2118, 'ep02': 1378, 'ep03': 2850, 'ep04': 2730, 'ep05': 2624,
        'ep06': 4690, 'ep07': 4440, 'ep08': 5020, 'ep09': 4568, 'ep10': 4390
    }

    results = []

    for file in instance_files:
        name = file.split('.')[0]
        print(f"\n{'='*40}")
        print(f"Processando instância: {name}")
        print(f"{'='*40}")

        instance = read_ep_instance(os.path.join(instances_dir, file))
        flavors = []
        times = []
        best_solutions = []
        mate_stats = []

        for run in range(runs):
            print(f"Execução {run+1}/{runs}")
            start_time = time.time()

            ga = EvolutionaryDanceEP(
                  instance,
                  pop_size=70, #diminuir a população aumenta numero médio de parceiros
                  mutation_rate=0.0097, #aumentar diminui a variação de fitness
                  alpha=0.0398,
                  max_iter=100,
                  max_mates_per_female=3,
                  low_status_mating_prob=0.68,
                  reverse_chimerism_prob=0.72,

                  status_decay_rate=0.36,
                  low_status_threshold=0.41,
                  status_penalty_exponent=50
            )

            best_sol, flavor = ga.evolve()
            elapsed = time.time() - start_time

            flavors.append(flavor)
            times.append(elapsed)
            best_solutions.append(best_sol)
            mate_stats.append(np.mean(ga.mate_counts))

            print(f"  Sabor: {flavor} | Tempo: {elapsed:.2f}s | Mates: {mate_stats[-1]:.2f}")

        # Estatísticas
        best_flavor = max(flavors)
        best_idx = flavors.index(best_flavor)
        best_sol = best_solutions[best_idx]
        avg_flavor = np.mean(flavors)
        std_flavor = np.std(flavors)
        avg_time = np.mean(times)
        avg_mates = np.mean(mate_stats)
        dev_bkv = 100 * (bkv[name] - best_flavor) / bkv[name] if bkv[name] != 0 else 0

        results.append({
            'instance': name,
            'best_flavor': best_flavor,
            'avg_flavor': avg_flavor,
            'std_flavor': std_flavor,
            'avg_time': avg_time,
            'avg_mates': avg_mates,
            'dev_bkv': dev_bkv,
            'solution': best_sol
        })

        # Salva melhor solução
        with open(os.path.join(output_dir, f"{name}_solution.txt"), 'w') as f:
            f.write(f"Instância: {name}\n")
            f.write(f"Sabor: {best_flavor}\n")
            f.write(f"Peso: {np.sum(instance['w'] * best_sol)}/{instance['W']}\n")
            f.write(f"Mates per female: {avg_mates:.2f}\n")
            f.write("\nSolução (ingredientes selecionados):\n")
            for i in range(instance['n']):
                if best_sol[i] == 1:
                    f.write(f"Ingrediente {i+1}: sabor={instance['t'][i]}, peso={instance['w'][i]}\n")

    # Salva resultados consolidados
    with open(os.path.join(output_dir, 'results_summary.csv'), 'w') as f:
        f.write("Instância,Melhor Sabor,Sabor Médio,Desvio Padrão,Tempo Médio (s),Mates Médios,Desvio BKV (%)\n")
        for res in results:
            f.write(f"{res['instance']},{res['best_flavor']},{res['avg_flavor']:.2f},{res['std_flavor']:.2f},{res['avg_time']:.2f},{res['avg_mates']:.2f},{res['dev_bkv']:.2f}\n")

    return results

# Execução do experimento
if __name__ == "__main__":
    INSTANCES_DIR = 'metaheuristicas/instances'
    OUTPUT_DIR = 'resultados_evolutionary_dance'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    results = run_evolutionary_dance_experiment(
        instances_dir=INSTANCES_DIR,
        output_dir=OUTPUT_DIR,
        runs=2
    )

    print("\nResumo dos Resultados:")
    for res in results:
        print(f"{res['instance']}: Sabor={res['best_flavor']} (Média: {res['avg_flavor']:.1f}±{res['std_flavor']:.1f}) | "
              f"Mates: {res['avg_mates']:.2f} | Desvio BKV: {res['dev_bkv']:.2f}%")


Processando instância: ep01
Execução 1/2
Gen 0: Best Fitness=1544.0 | Status Diff=-0.088 | Chimerism=0.000 | Avg Mates=1.77
Gen 10: Best Fitness=1646.3 | Status Diff=0.031 | Chimerism=0.000 | Avg Mates=2.22
Gen 20: Best Fitness=1736.9 | Status Diff=-0.016 | Chimerism=0.000 | Avg Mates=1.85
Gen 30: Best Fitness=1845.1 | Status Diff=0.052 | Chimerism=0.000 | Avg Mates=2.04
Gen 40: Best Fitness=1913.9 | Status Diff=0.056 | Chimerism=0.000 | Avg Mates=1.93
Gen 50: Best Fitness=1952.3 | Status Diff=-0.063 | Chimerism=0.000 | Avg Mates=2.00
Gen 60: Best Fitness=2021.2 | Status Diff=-0.074 | Chimerism=0.000 | Avg Mates=1.73
Gen 70: Best Fitness=2021.2 | Status Diff=-0.019 | Chimerism=0.000 | Avg Mates=1.87
Gen 80: Best Fitness=2072.4 | Status Diff=-0.062 | Chimerism=0.000 | Avg Mates=1.90
Gen 90: Best Fitness=2072.4 | Status Diff=0.043 | Chimerism=0.000 | Avg Mates=1.76

Melhor solução encontrada em 56.16s:
  Sabor total: 1941
  Peso total: 1691/1800
  Fitness: 2171.1
  Mates per female: 1.8

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-1994998838.py", line 705, in <cell line: 0>
    results = run_evolutionary_dance_experiment(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1-1994998838.py", line 649, in run_evolutionary_dance_experiment
    best_sol, flavor = ga.evolve()
                       ^^^^^^^^^^^
  File "/tmp/ipython-input-1-1994998838.py", line 469, in evolve
    fitness = self.evaluate_fitness()
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1-1994998838.py", line None, in evaluate_fitness
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._rend

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import time
from collections import defaultdict

class EvolutionaryDanceEP:
    def __init__(self, instance, pop_size=200, mutation_rate=0.2, alpha=0.31, max_iter=1000,
                 max_mates_per_female=3, low_status_mating_prob=0.15, reverse_chimerism_prob=0.25):
        # Parâmetros do problema
        self.instance = instance
        self.n = instance['n']
        self.W = instance['W']
        self.w = np.array(instance['w'])
        self.t = np.array(instance['t'])
        self.neighbors = self.build_neighbors()

        # Novos parâmetros biológicos
        self.max_mates_per_female = max_mates_per_female
        self.low_status_mating_prob = low_status_mating_prob
        self.reverse_chimerism_prob = reverse_chimerism_prob

        # Parâmetros do algoritmo
        self.pop_size = pop_size
        self.mutation_rate = mutation_rate
        self.alpha = alpha  # Taxa de microquimerismo
        self.max_iter = max_iter

        # Inicialização da população
        self.population = self.initialize_population()
        self.gender = np.array(['F'] * (pop_size // 2) + ['M'] * (pop_size // 2))
        self.status = np.zeros(pop_size)  # Nível de status/recurso
        self.dance_style = np.random.rand(pop_size, 5)  # Padrões de dança
        self.chimeric_dna = [set() for _ in range(pop_size)]  # Rastreamento de microquimerismo
        self.epigenetic_marks = [np.zeros(self.n) for _ in range(pop_size)]  # Marcas epigenéticas
        self.fitness_history = []
        self.best_solution = None
        self.best_fitness = -np.inf
        self.mate_counts = []  # Para armazenar estatísticas de acasalamento

        # Inicializar atributos
        self.initialize_attributes()

    def build_neighbors(self):
        """Constrói lista de vizinhos para cada ingrediente"""
        neighbors = [[] for _ in range(self.n)]
        for j, k in self.instance['incompatible_pairs']:
            neighbors[j].append(k)
            neighbors[k].append(j)
        return neighbors

    def initialize_population(self):
        """Inicializa população com soluções viáveis"""
        population = []
        while len(population) < self.pop_size:
            sol = np.zeros(self.n, dtype=int)
            weight = 0

            # Adiciona ingredientes aleatórios respeitando restrições
            indices = list(range(self.n))
            random.shuffle(indices)

            for i in indices:
                if weight + self.w[i] > self.W:
                    continue

                # Verifica conflitos
                conflict = False
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        conflict = True
                        break

                if not conflict and random.random() < 0.7:
                    sol[i] = 1
                    weight += self.w[i]

            population.append(sol)
        return np.array(population)

    def initialize_attributes(self):
        """Inicializa status e dança com base na solução"""
        # Calcula fitness para cada indivíduo
        fitness = []
        for sol in self.population:
            flavor = np.sum(self.t * sol)
            weight = np.sum(self.w * sol)

            # Penaliza soluções inviáveis
            penalty = 0
            if weight > self.W:
                penalty = (weight - self.W) * 100

            for i in range(self.n):
                if sol[i] == 1:
                    for j in self.neighbors[i]:
                        if sol[j] == 1:
                            penalty += 100
                            break

            fitness.append(flavor - penalty)

        fitness = np.array(fitness)
        self.status = (fitness - np.min(fitness)) / (np.max(fitness) - np.min(fitness) + 1e-10)

        # Atualiza melhor solução
        max_idx = np.argmax(fitness)
        if fitness[max_idx] > self.best_fitness:
            self.best_fitness = fitness[max_idx]
            self.best_solution = self.population[max_idx].copy()

        # Estilo de dança baseado na solução
        for i in range(self.pop_size):
            sol = self.population[i]
            # Características: proporção de ingredientes selecionados em grupos
            group_size = max(1, self.n // 5)
            for g in range(5):
                start = g * group_size
                end = min((g+1) * group_size, self.n)
                self.dance_style[i, g] = np.mean(sol[start:end])

    def evaluate_fitness(self):
        """Calcula fitness baseado na função objetivo e penalidades"""
        fitness = []
        for i, sol in enumerate(self.population):
            flavor = np.sum(self.t * sol)
            weight = np.sum(self.w * sol)

            # Penaliza soluções inviáveis
            penalty = 0
            if weight > self.W:
                penalty = (weight - self.W) * 100

            # Verifica incompatibilidades
            for j in range(self.n):
                if sol[j] == 1:
                    for k in self.neighbors[j]:
                        if sol[k] == 1:
                            penalty += 100
                            break

            # Aplica efeito epigenético (genes ativados têm maior contribuição)
            if i < len(self.epigenetic_marks):  # Verificação de segurança
                activated_flavor = np.sum(self.t * sol * (1 + 0.2 * self.epigenetic_marks[i]))
                flavor = max(flavor, activated_flavor)

            fitness.append(flavor - penalty)

        fitness = np.array(fitness)

        # Atualiza melhor solução global
        max_idx = np.argmax(fitness)
        if fitness[max_idx] > self.best_fitness:
            self.best_fitness = fitness[max_idx]
            self.best_solution = self.population[max_idx].copy()

        self.fitness_history.append(np.max(fitness))
        return fitness

    def female_dance_attraction(self, female_id, male_ids):
        """Calcula atração gerada pela dança da fêmea"""
        female_dance = self.dance_style[female_id]
        reactions = []

        for mid in male_ids:
            # Verificação de segurança para índices válidos
            if mid >= len(self.dance_style) or mid < 0:
                reactions.append(0)
                continue

            # Macho responde à dança baseado em seu status
            male_status = self.status[mid]

            # Similaridade de movimento (distância cosseno)
            male_dance = self.dance_style[mid]
            norm_female = np.linalg.norm(female_dance)
            norm_male = np.linalg.norm(male_dance)

            if norm_female > 1e-10 and norm_male > 1e-10:
                dance_similarity = np.dot(female_dance, male_dance) / (norm_female * norm_male)
            else:
                dance_similarity = 0.0

            # Machos de alto status são mais exigentes
            attraction = male_status * dance_similarity

            # Adicionar componente aleatória (10-20%)
            randomness = 0.8 + 0.4 * random.random()
            reactions.append(attraction * randomness)

        return np.array(reactions)

    def mate_selection(self):
        """Seleção hierárquica com fêmeas podendo ter múltiplos parceiros"""
        female_ids = np.where(self.gender == 'F')[0]
        male_ids = np.where(self.gender == 'M')[0]

        # Filtra índices válidos
        female_ids = female_ids[(female_ids >= 0) & (female_ids < self.pop_size)]
        male_ids = male_ids[(male_ids >= 0) & (male_ids < self.pop_size)]

        if len(female_ids) == 0 or len(male_ids) == 0:
            return []

        # Fêmeas dançam para todos os machos simultaneamente
        attraction_matrix = np.zeros((len(female_ids), len(male_ids)))
        for i, fid in enumerate(female_ids):
            attraction_matrix[i] = self.female_dance_attraction(fid, male_ids)

        # Machos escolhem fêmeas baseado na atração percebida
        pairs = []
        female_mate_counts = defaultdict(int)  # Contagem de parceiros por fêmea

        # Machos de alto status escolhem primeiro
        male_status = self.status[male_ids]
        sorted_male_indices = np.argsort(male_status)[::-1]

        for local_midx in sorted_male_indices:
            mid = male_ids[local_midx]  # ID global do macho

            # Fêmeas disponíveis (que ainda não atingiram o máximo de parceiros)
            available_females = [i for i, fid in enumerate(female_ids)
                                if female_mate_counts[fid] < self.max_mates_per_female]

            if not available_females:
                continue

            # Escolhe a fêmea mais atraente disponível
            attraction_values = attraction_matrix[available_females, local_midx]
            best_fidx_in_available = np.argmax(attraction_values)
            fid = female_ids[available_females[best_fidx_in_available]]

            pairs.append((fid, mid))
            female_mate_counts[fid] += 1

        # Fêmeas podem escolher machos de baixo status adicionalmente
        for fid in female_ids:
            if random.random() < self.low_status_mating_prob and female_mate_counts[fid] < self.max_mates_per_female:
                # Seleciona machos de baixo status não pareados com esta fêmea
                current_mates = [m for f, m in pairs if f == fid]
                low_status_males = [mid for mid in male_ids
                                   if mid not in current_mates and self.status[mid] < 0.5]

                if low_status_males:
                    chosen_male = random.choice(low_status_males)
                    pairs.append((fid, chosen_male))
                    female_mate_counts[fid] += 1

        return pairs

    def microchimerism(self, recipient_id, donor_id):
        """Transferência de DNA entre indivíduos"""
        # Verificação de índices válidos
        if recipient_id >= self.pop_size or donor_id >= self.pop_size:
            return

        # Selecionar genes para transferência
        transfer_count = max(1, int(self.alpha * self.n))
        gene_idx = random.sample(range(self.n), transfer_count)

        # Incorporar genes ao DNA do recipiente
        for idx in gene_idx:
            self.population[recipient_id][idx] = self.population[donor_id][idx]
            # Atualizar marca epigenética (50% de chance de herdar a marca)
            if random.random() < 0.5:
                self.epigenetic_marks[recipient_id][idx] = self.epigenetic_marks[donor_id][idx]

        # Registrar microquimerismo
        self.chimeric_dna[recipient_id].update(gene_idx)

    def reverse_microchimerism(self, female_id, male_id):
        """Fêmea edita genes de machos de baixo status"""
        # Verificação de índices válidos
        if female_id >= self.pop_size or male_id >= self.pop_size:
            return

        # Genes que a fêmea pode transferir para o macho
        transfer_count = max(1, int(self.alpha * self.n))
        gene_idx = random.sample(range(self.n), transfer_count)

        for idx in gene_idx:
            # A fêmea transfere seu gene com 60% de chance
            if random.random() < 0.6:
                self.population[male_id][idx] = self.population[female_id][idx]
                # Herança epigenética mais forte (70% de chance)
                if random.random() < 0.7:
                    self.epigenetic_marks[male_id][idx] = self.epigenetic_marks[female_id][idx]

        # Registrar no macho
        self.chimeric_dna[male_id].update(gene_idx)

    def repair_solution(self, sol):
        """Repara solução para torná-la viável usando abordagem gulosa"""
        weight = np.sum(self.w * sol)

        # Lista de ingredientes selecionados
        selected = np.where(sol == 1)[0]

        # Remove ingredientes se peso exceder capacidade
        while weight > self.W and len(selected) > 0:
            # Encontra o ingrediente com menor razão sabor/peso
            min_ratio = float('inf')
            idx_to_remove = -1
            for i in selected:
                ratio = self.t[i] / self.w[i] if self.w[i] > 0 else float('inf')
                if ratio < min_ratio:
                    min_ratio = ratio
                    idx_to_remove = i

            sol[idx_to_remove] = 0
            weight -= self.w[idx_to_remove]
            selected = np.setdiff1d(selected, [idx_to_remove])

        # Remove conflitos de incompatibilidade
        for i in range(self.n):
            if sol[i] == 1:
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        # Remove o que tem menor sabor
                        if self.t[i] < self.t[j]:
                            sol[i] = 0
                        else:
                            sol[j] = 0
                        break

        # Tenta adicionar ingredientes sem conflitos (busca local)
        indices = list(range(self.n))
        # Ordena por sabor/peso decrescente
        indices.sort(key=lambda i: self.t[i] / self.w[i] if self.w[i] > 0 else 0, reverse=True)

        for i in indices:
            if sol[i] == 0 and weight + self.w[i] <= self.W:
                conflict = False
                for j in self.neighbors[i]:
                    if sol[j] == 1:
                        conflict = True
                        break
                if not conflict:
                    sol[i] = 1
                    weight += self.w[i]

        return sol

    def epigenetic_inheritance(self, mother, father, child_dna):
        """Herança epigenética com influência parental"""
        # Herança: 70% de chance de herdar a marca da mãe, 30% do pai
        if mother < len(self.epigenetic_marks) and father < len(self.epigenetic_marks):
            if random.random() < 0.7:
                child_epigenetic = self.epigenetic_marks[mother].copy()
            else:
                child_epigenetic = self.epigenetic_marks[father].copy()
        else:
            child_epigenetic = np.zeros(self.n)

        # Modificação epigenética baseada no ambiente (solução)
        for i in range(self.n):
            if child_dna[i] == 1 and random.random() < 0.3:
                child_epigenetic[i] = min(1.0, child_epigenetic[i] + 0.2)
            elif child_dna[i] == 0 and random.random() < 0.2:
                child_epigenetic[i] = max(0.0, child_epigenetic[i] - 0.1)

        return child_epigenetic

    def reproduce(self, pairs):
        new_population = []
        new_chimeric = []
        new_epigenetic = []
        new_gender = []

        # Cada par pode produzir filhos
        for mother, father in pairs:
            for _ in range(2):  # Cada par produz 2 filhos
                # Determinar gênero
                gender = 'F' if random.random() < 0.5 else 'M'
                new_gender.append(gender)

                # Herança genética (crossover uniforme com viés materno)
                child_dna = np.zeros(self.n, dtype=int)
                for i in range(self.n):
                    if random.random() < 0.6:  # 60% de chance de herdar da mãe
                        child_dna[i] = self.population[mother][i]
                    else:
                        child_dna[i] = self.population[father][i]

                # Microquimerismo materno
                if mother < len(self.chimeric_dna) and self.chimeric_dna[mother]:
                    chimeric_genes = random.sample(
                        list(self.chimeric_dna[mother]),
                        min(3, len(self.chimeric_dna[mother])))
                    for gene in chimeric_genes:
                        child_dna[gene] = self.population[mother][gene]

                # Mutação
                for i in range(self.n):
                    if random.random() < self.mutation_rate:
                        child_dna[i] = 1 - child_dna[i]

                # Repara solução
                child_dna = self.repair_solution(child_dna)

                new_population.append(child_dna)
                new_chimeric.append(set())

                # Herança epigenética
                child_epigenetic = self.epigenetic_inheritance(mother, father, child_dna)
                new_epigenetic.append(child_epigenetic)

        # Preencher população restante com as melhores soluções
        if len(new_population) < self.pop_size:
            fitness = self.evaluate_fitness()
            best_indices = np.argsort(fitness)[-(self.pop_size - len(new_population)):]
            for idx in best_indices:
                if idx < len(self.population):
                    new_population.append(self.population[idx].copy())
                    new_chimeric.append(self.chimeric_dna[idx].copy() if idx < len(self.chimeric_dna) else set())
                    new_gender.append(self.gender[idx])
                    new_epigenetic.append(self.epigenetic_marks[idx].copy() if idx < len(self.epigenetic_marks) else np.zeros(self.n))

        # Atualizar população
        self.population = np.array(new_population)
        self.chimeric_dna = new_chimeric
        self.gender = np.array(new_gender)
        self.epigenetic_marks = new_epigenetic

    def evolve(self):
        start_time = time.time()
        status_diff = []
        chimeric_levels = []

        for gen in range(self.max_iter):
            fitness = self.evaluate_fitness()
            pairs = self.mate_selection()

            # Coletar estatísticas de acasalamento
            female_mates = defaultdict(int)
            for f, m in pairs:
                female_mates[f] += 1
            if female_mates:
                self.mate_counts.append(np.mean(list(female_mates.values())))
            else:
                self.mate_counts.append(0)

            # Aplicar microquimerismo e microquimerismo reverso
            for mother, father in pairs:
                # Microquimerismo tradicional (macho -> fêmea)
                self.microchimerism(mother, father)

                # Microquimerismo reverso (fêmea -> macho) para machos de baixo status
                if father < len(self.status) and self.status[father] < 0.4 and random.random() < self.reverse_chimerism_prob:
                    self.reverse_microchimerism(mother, father)

            self.reproduce(pairs)
            self.initialize_attributes()  # Atualizar status e dança

            # Coletar dados para análise
            if pairs:
                f_status = [self.status[f] for f, _ in pairs if f < len(self.status)]
                m_status = [self.status[m] for _, m in pairs if m < len(self.status)]
                status_diff.append(np.mean(m_status) - np.mean(f_status) if f_status and m_status else 0)
            else:
                status_diff.append(0)

            # Calcular nível de microquimerismo
            if self.chimeric_dna:
                chimeric_levels.append(np.mean([len(dna) for dna in self.chimeric_dna if dna is not None]) / self.n)
            else:
                chimeric_levels.append(0)

            # Log de progresso
            if gen % 10 == 0:
                print(f"Gen {gen}: Best Fitness={self.best_fitness:.1f} | "
                      f"Status Diff={status_diff[-1]:.3f} | "
                      f"Chimerism={chimeric_levels[-1]:.3f} | "
                      f"Avg Mates={self.mate_counts[-1]:.2f}")

        # Resultado final
        elapsed = time.time() - start_time
        flavor = np.sum(self.t * self.best_solution)
        weight = np.sum(self.w * self.best_solution)

        print(f"\nMelhor solução encontrada em {elapsed:.2f}s:")
        print(f"  Sabor total: {flavor}")
        print(f"  Peso total: {weight}/{self.W}")
        print(f"  Fitness: {self.best_fitness:.1f}")
        print(f"  Mates per female: {np.mean(self.mate_counts):.2f}")

        return self.best_solution, flavor

    def plot_results(self):
        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(self.fitness_history, 'g-')
        plt.title('Evolução da Aptidão Máxima')
        plt.xlabel('Geração')
        plt.ylabel('Fitness')
        plt.grid(alpha=0.3)

        plt.subplot(2, 2, 2)
        if self.chimeric_dna:
            chimeric_levels = [len(dna)/self.n for dna in self.chimeric_dna if dna is not None]
            plt.hist(chimeric_levels, bins=20, color='blue', alpha=0.7) if chimeric_levels else None
            plt.title('Distribuição de Microquimerismo')
            plt.xlabel('Frações de DNA Incorporado')
            plt.ylabel('Frequência')

        plt.subplot(2, 2, 3)
        female_indices = np.where(self.gender == 'F')[0]
        male_indices = np.where(self.gender == 'M')[0]
        if female_indices.size > 0 and male_indices.size > 0:
            plt.scatter(self.status[female_indices], self.status[male_indices], alpha=0.6)
            plt.title('Correlação de Status entre Gêneros')
            plt.xlabel('Status Feminino')
            plt.ylabel('Status Masculino')

        plt.subplot(2, 2, 4)
        if self.mate_counts:
            plt.hist(self.mate_counts, bins=np.arange(0, self.max_mates_per_female+2)-0.5,
                    rwidth=0.8, color='purple')
            plt.title('Distribuição de Parceiros por Fêmea')
            plt.xlabel('Número de Parceiros')
            plt.ylabel('Frequência')

        plt.tight_layout()
        plt.savefig('evolutionary_dance_ep_results.png')
        plt.show()

# Função para ler instâncias
def read_ep_instance(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    blocks = []
    current_block = []
    for line in lines:
        if not line:
            if current_block:
                blocks.append(current_block)
                current_block = []
        else:
            current_block.append(line)
    if current_block:
        blocks.append(current_block)

    n, num_incompatible, W = map(int, blocks[0][0].split())

    t = []
    for line in blocks[1]:
        t.extend(map(int, line.split()))

    w = []
    for line in blocks[2]:
        w.extend(map(int, line.split()))

    incompatible_pairs = []
    if len(blocks) >= 4:
        for line in blocks[3]:
            j, k = map(int, line.split())
            incompatible_pairs.append((j-1, k-1))

    return {
        'n': n,
        'num_incompatible': num_incompatible,
        'W': W,
        't': t,
        'w': w,
        'incompatible_pairs': incompatible_pairs
    }

# Experimento principal com novos parâmetros
def run_evolutionary_dance_experiment(instances_dir, output_dir, runs=10):
    instance_files = sorted([f for f in os.listdir(instances_dir) if f.endswith('.dat')])
    bkv = {
        'ep01': 2118, 'ep02': 1378, 'ep03': 2850, 'ep04': 2730, 'ep05': 2624,
        'ep06': 4690, 'ep07': 4440, 'ep08': 5020, 'ep09': 4568, 'ep10': 4390
    }

    results = []

    for file in instance_files:
        name = file.split('.')[0]
        print(f"\n{'='*40}")
        print(f"Processando instância: {name}")
        print(f"{'='*40}")

        instance = read_ep_instance(os.path.join(instances_dir, file))
        flavors = []
        times = []
        best_solutions = []
        mate_stats = []

        for run in range(runs):
            print(f"Execução {run+1}/{runs}")
            start_time = time.time()

            ga = EvolutionaryDanceEP(
                instance,
                pop_size=130,
                mutation_rate=0.001,
                alpha=0.0012,
                max_iter=70,
                max_mates_per_female=5,
                low_status_mating_prob=0.3,
                reverse_chimerism_prob=0.5
            )

            best_sol, flavor = ga.evolve()
            elapsed = time.time() - start_time

            flavors.append(flavor)
            times.append(elapsed)
            best_solutions.append(best_sol)
            mate_stats.append(np.mean(ga.mate_counts))

            print(f"  Sabor: {flavor} | Tempo: {elapsed:.2f}s | Mates: {mate_stats[-1]:.2f}")

        # Estatísticas
        best_flavor = max(flavors)
        best_idx = flavors.index(best_flavor)
        best_sol = best_solutions[best_idx]
        avg_flavor = np.mean(flavors)
        std_flavor = np.std(flavors)
        avg_time = np.mean(times)
        avg_mates = np.mean(mate_stats)
        dev_bkv = 100 * (bkv[name] - best_flavor) / bkv[name] if bkv[name] != 0 else 0

        results.append({
            'instance': name,
            'best_flavor': best_flavor,
            'avg_flavor': avg_flavor,
            'std_flavor': std_flavor,
            'avg_time': avg_time,
            'avg_mates': avg_mates,
            'dev_bkv': dev_bkv,
            'solution': best_sol
        })

        # Salva melhor solução
        with open(os.path.join(output_dir, f"{name}_solution.txt"), 'w') as f:
            f.write(f"Instância: {name}\n")
            f.write(f"Sabor: {best_flavor}\n")
            f.write(f"Peso: {np.sum(instance['w'] * best_sol)}/{instance['W']}\n")
            f.write(f"Mates per female: {avg_mates:.2f}\n")
            f.write("\nSolução (ingredientes selecionados):\n")
            for i in range(instance['n']):
                if best_sol[i] == 1:
                    f.write(f"Ingrediente {i+1}: sabor={instance['t'][i]}, peso={instance['w'][i]}\n")

    # Salva resultados consolidados
    with open(os.path.join(output_dir, 'results_summary.csv'), 'w') as f:
        f.write("Instância,Melhor Sabor,Sabor Médio,Desvio Padrão,Tempo Médio (s),Mates Médios,Desvio BKV (%)\n")
        for res in results:
            f.write(f"{res['instance']},{res['best_flavor']},{res['avg_flavor']:.2f},{res['std_flavor']:.2f},{res['avg_time']:.2f},{res['avg_mates']:.2f},{res['dev_bkv']:.2f}\n")

    return results

# Execução do experimento
if __name__ == "__main__":
    INSTANCES_DIR = 'metaheuristicas/instances'
    OUTPUT_DIR = 'resultados_evolutionary_dance'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    results = run_evolutionary_dance_experiment(
        instances_dir=INSTANCES_DIR,
        output_dir=OUTPUT_DIR,
        runs=15
    )

    print("\nResumo dos Resultados:")
    for res in results:
        print(f"{res['instance']}: Sabor={res['best_flavor']} (Média: {res['avg_flavor']:.1f}±{res['std_flavor']:.1f}) | "
              f"Mates: {res['avg_mates']:.2f} | Desvio BKV: {res['dev_bkv']:.2f}%")


Processando instância: ep01
Execução 1/15
Gen 0: Best Fitness=1675.0 | Status Diff=0.013 | Chimerism=0.000 | Avg Mates=1.76
Gen 10: Best Fitness=1684.9 | Status Diff=-0.058 | Chimerism=0.000 | Avg Mates=1.77
Gen 20: Best Fitness=1980.4 | Status Diff=-0.008 | Chimerism=0.000 | Avg Mates=2.08
Gen 30: Best Fitness=2089.5 | Status Diff=0.057 | Chimerism=0.000 | Avg Mates=1.67
Gen 40: Best Fitness=2156.4 | Status Diff=-0.025 | Chimerism=0.000 | Avg Mates=1.67
Gen 50: Best Fitness=2223.1 | Status Diff=-0.007 | Chimerism=0.000 | Avg Mates=1.62
Gen 60: Best Fitness=2223.1 | Status Diff=-0.022 | Chimerism=0.000 | Avg Mates=1.70

Melhor solução encontrada em 47.49s:
  Sabor total: 1884
  Peso total: 1614/1800
  Fitness: 2240.5
  Mates per female: 1.81
  Sabor: 1884 | Tempo: 47.93s | Mates: 1.81
Execução 2/15
Gen 0: Best Fitness=1613.0 | Status Diff=-0.094 | Chimerism=0.000 | Avg Mates=1.93
Gen 10: Best Fitness=1760.4 | Status Diff=-0.080 | Chimerism=0.000 | Avg Mates=1.79
Gen 20: Best Fitness=1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-20-972463449.py", line 664, in <cell line: 0>
    results = run_evolutionary_dance_experiment(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-20-972463449.py", line 608, in run_evolutionary_dance_experiment
    best_sol, flavor = ga.evolve()
                       ^^^^^^^^^^^
  File "/tmp/ipython-input-20-972463449.py", line 453, in evolve
    self.reproduce(pairs)
  File "/tmp/ipython-input-20-972463449.py", line 400, in reproduce
    child_dna = self.repair_solution(child_dna)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-20-972463449.py", line None, in repair_solution
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/